**Loads baseline Showcase data and processes into format that is suitable for input into XGBoost.**

Baseline data is exported in *scripts/extract_ukb_baseline.py*

Note: field is defined as a high-level field, such as BMI. Columns are defined as all the columns provided in the dataset that relate to that field (i.e. the multiple instances/arrays for each field) - e.g. BP Measure array 0, BP Measure array 1 etc... There may be (and indeed often are) multiple columns for each field, but not multiple fields for each column.

In [1]:
%load_ext autoreload
%autoreload 2
    
import pandas as pd
import numpy as np
import sys
sys.path.append("../src")
from processing_utils import display_included_cats, remove_cat_cols, get_cat_and_downstream, display_arrayed_fields, remove_fields

In [2]:
data = pd.read_feather("../data/processed/all_showcase_baseline.feather")

# Select categories
Not all categories captured by baseline extraction are relevant. Review these and remove irrelevant.

## View all included categories
These should all be baseline categories, as selected from the master showcase dataset in *scripts/extract_ukb_baseline.py*

In [3]:
cat_tree, pretty_df = display_included_cats(data.columns[1:])
pd.set_option('display.max_rows', None)
pretty_df

Empty DataFrame
Columns: []
Index: [(Assessment centre (100000), Recruitment (100021), Reception (100024) ✅, , , ), (Assessment centre (100000), Recruitment (100021), Consent (100023) ✅, , , ), (Assessment centre (100000), Recruitment (100021), Conclusion (100022), , , ), (Assessment centre (100000), Recruitment (100021), Consent for imaging (100119), , , ), (Assessment centre (100000), Touchscreen (100025), Sociodemographics (100062), Household (100066) ✅, , ), (Assessment centre (100000), Touchscreen (100025), Sociodemographics (100062), Employment (100064) ✅, , ), (Assessment centre (100000), Touchscreen (100025), Sociodemographics (100062), Education (100063) ✅, , ), (Assessment centre (100000), Touchscreen (100025), Sociodemographics (100062), Ethnicity (100065) ✅, , ), (Assessment centre (100000), Touchscreen (100025), Sociodemographics (100062), Other sociodemographic factors (100067) ✅, , ), (Assessment centre (100000), Touchscreen (100025), Lifestyle and environment (100050), Physical activity (100054) ✅, MET Scores (54) ✅, ), (Assessment centre (100000), Touchscreen (100025), Lifestyle and environment (100050), Electronic device use (100053) ✅, , ), (Assessment centre (100000), Touchscreen (100025), Lifestyle and environment (100050), Sleep (100057) ✅, , ), (Assessment centre (100000), Touchscreen (100025), Lifestyle and environment (100050), Smoking (100058) ✅, , ), (Assessment centre (100000), Touchscreen (100025), Lifestyle and environment (100050), Diet (100052) ✅, , ), (Assessment centre (100000), Touchscreen (100025), Lifestyle and environment (100050), Alcohol (100051) ✅, , ), (Assessment centre (100000), Touchscreen (100025), Lifestyle and environment (100050), Sun exposure (100055) ✅, , ), (Assessment centre (100000), Touchscreen (100025), Lifestyle and environment (100050), Sexual factors (100056) ✅, , ), (Assessment centre (100000), Touchscreen (100025), Early life factors (100033) ✅, , , ), (Assessment centre (100000), Touchscreen (100025), Family history (100034) ✅, , , ), (Assessment centre (100000), Touchscreen (100025), Psychosocial factors (100059), Social support (100061) ✅, , ), (Assessment centre (100000), Touchscreen (100025), Psychosocial factors (100059), Mental health (100060) ✅, , ), (Assessment centre (100000), Touchscreen (100025), Health and medical history (100036), Eyesight (100041) ✅, , ), (Assessment centre (100000), Touchscreen (100025), Health and medical history (100036), Mouth (100046) ✅, , ), (Assessment centre (100000), Touchscreen (100025), Health and medical history (100036), General health (100042) ✅, , ), (Assessment centre (100000), Touchscreen (100025), Health and medical history (100036), Breathing (100037) ✅, , ), (Assessment centre (100000), Touchscreen (100025), Health and medical history (100036), Claudication and peripheral artery disease (100038) ✅, , ), (Assessment centre (100000), Touchscreen (100025), Health and medical history (100036), Pain (100048) ✅, , ), (Assessment centre (100000), Touchscreen (100025), Health and medical history (100036), Chest pain (100039) ✅, , ), (Assessment centre (100000), Touchscreen (100025), Health and medical history (100036), Cancer screening (100040) ✅, , ), (Assessment centre (100000), Touchscreen (100025), Health and medical history (100036), Operations (100047) ✅, , ), (Assessment centre (100000), Touchscreen (100025), Health and medical history (100036), Medical conditions (100044) ✅, , ), (Assessment centre (100000), Touchscreen (100025), Health and medical history (100036), Medication (100045) ✅, , ), (Assessment centre (100000), Touchscreen (100025), Health and medical history (100036), Hearing (100043) ✅, , ), (Assessment centre (100000), Touchscreen (100025), Sex-specific factors (100068), Male-specific factors (100070) ✅, , ), (Assessment centre (100000), Touchscreen (100025), Sex-specific factors (100068), Female-specific factors (100069) ✅, , ), (Assessment centre (100000), Cognitive function (100026), Reaction time (

## Remove undesired categories

In [4]:
print(f"Before pruning: {len(data.columns[1:])} columns \n")

# Remove health-related outcomes. To be processed separately.
pruned_cols = remove_cat_cols(columns=data.columns[1:], categories=get_cat_and_downstream(100091, cat_tree))

# Remove kidney-derived measures from imaging: kidney fusion field (21164) wrongly labelled as instance 0.
pruned_cols = remove_cat_cols(columns=pruned_cols, categories=[159])

# Remove procedural metrics related to assessment centre
pruned_cols = remove_cat_cols(columns=pruned_cols, categories=get_cat_and_downstream(100004, cat_tree))

# Remove biological samples processing
pruned_cols = remove_cat_cols(columns=pruned_cols, categories=[9081, 18518, 1307, 221, 222, 148])

# Remove biological sample inventory
pruned_cols = remove_cat_cols(columns=pruned_cols, categories=get_cat_and_downstream(100084, cat_tree))

# Remove additional physical activity measurements (not baseline)
pruned_cols = remove_cat_cols(columns=pruned_cols, categories=get_cat_and_downstream(1008, cat_tree))

# Remove any online follow-up (not baseline)
pruned_cols = remove_cat_cols(columns=pruned_cols, categories=get_cat_and_downstream(100089, cat_tree))

# Remove genotyping: indicators of data availability (genotypes provided as separate bulk files)
pruned_cols = remove_cat_cols(columns=pruned_cols, categories=get_cat_and_downstream(263, cat_tree))
pruned_cols = remove_cat_cols(columns=pruned_cols, categories=[100319])

print(f"\n= {len(pruned_cols)} columns after pruning")

data = data[['eid'] + pruned_cols]

Before pruning: 12879 columns 

Removed 4497 columns
Removed 1 columns
Removed 36 columns
Removed 544 columns
Removed 13 columns
Removed 194 columns
Removed 3539 columns
Removed 85 columns
Removed 2 columns

= 3968 columns after pruning


# Arrayed fields
Arrayed fields are unsuitable for feeding straight into XGBoost. Some of these are too granular so can be removed altogether (e.g. ECG), while others can be engineered into a suitable format (e.g. one-hot-encoding of multi-choice fields).

## Identify all arrayed fields.

In [5]:
arrayed_fields = display_arrayed_fields(pruned_cols)
arrayed_fields

Field ID  \
Primary category                              Field title                                                    
Blood pressure (100011)                       Systolic blood pressure, manual reading                   93   
                                              Diastolic blood pressure, manual reading                  94   
                                              Pulse rate (during blood-pressure measurement)            95   
                                              Time since interview start at which blood press...        96   
                                              Pulse rate, automated reading                            102   
                                              Diastolic blood pressure, automated reading             4079   
                                              Systolic blood pressure, automated reading              4080   
                                              Method of measuring blood pressure                      4081   
Blood sample collection (100002)              Time blood sample collected                             3166   
                                              Blood sample #, note contents                          20049   
Diet (100052)                                 Never eat eggs, dairy, wheat, sugar                     6144   
                                              Never eat eggs, dairy, wheat, sugar (pilot)            10855   
ECG during exercise (100012)                  ECG, heart rate                                         5983   
                                              ECG, load                                               5984   
                                              Bicycle speed                                           5985   
                                              ECG, phase time                                         5986   
                                              ECG, trend phase name                                   5987   
                                              ECG, stage name                                         5988   
                                              ECG, phase name                                         5991   
                                              ECG, phase duration                                     5992   
                                              ECG, number of stages in a phase                        5993   
Education (100063)                            Qualifications                                          6138   
                                              Qualifications (pilot)                                 10722   
Employment (100064)                           Current employment status                               6142   
                                              Transport type for commuting to job workplace           6143   
Eyesight (100041)                             Reason for glasses/contact lenses                       6147   
                                              Eye problems/disorders                                  6148   
Family history (100034)                       Illnesses of father                                    20107   
                                              Illnesses of mother                                    20110   
                                              Illnesses of siblings                                  20111   
                                              Illnesses of adopted father                            20112   
                                              Illnesses of adopted mother                            20113   
                                              Illnesses of adopted siblings                          20114   
Hearing test (100049)                         Triplet played (left)                                   4229   
                                              Signal-to-noise-ratio (SNR) of triplet (left)           4230   
                            

## Remove highly granular fields with large arrays.
Some of these (e.g. those relating to medical history) need to be processed separately.

In [6]:
# Select field IDs of the following arrayed categories and remove. Note that this is not removing the 
# whole category (as done previously) but just the arrayed fields within each category selected for removal, as listed above.

arrayed_categories = ['ECG during exercise (100012)', 'Hearing test (100049)', 'Medical conditions (100074)', 
                      'Medications (100075)', 'Numeric memory (100029)', 'Operations (100076)', 'Pairs matching (100030)',
                      'Polygenic Risk Scores (300)', 'Reaction time (100032)', 'Refractometer 1 (100014)',
                      'Retinal optical coherence tomography (100016)', 'Visual acuity (100017)', 'Word production (100077)']

for category in arrayed_categories:
    # Use remove_fields() with field IDs for arrayed fields from table above as input for removal.
    pruned_cols = remove_fields(columns=pruned_cols, fields_to_remove=arrayed_fields.loc[(category), 'Field ID'].tolist())

print(f"\n= {len(pruned_cols)} columns after pruning")

arrayed_fields = display_arrayed_fields(pruned_cols)

data = data[['eid'] + pruned_cols]

Removed 691 columns
Removed 300 columns
Removed 200 columns
Removed 48 columns
Removed 176 columns
Removed 160 columns
Removed 29 columns
Removed 4 columns
Removed 128 columns
Removed 388 columns
Removed 18 columns
Removed 160 columns
Removed 51 columns

= 1615 columns after pruning


## One-hot-encode multiple choice fields
There is no direct 1:1 relationship between a field's arrayed column and a specific multiple-choice option for that field: order in array columns seems to be order in which user picked options. As the same option may be present in different columns for each user, these must be aggregated across all columns and used to create a one-hot-encoding.

In [7]:
arrayed_fields = display_arrayed_fields(pruned_cols)
multi_choice = arrayed_fields[arrayed_fields['Field type'] == 'Multi choice']

In [8]:
multi_choice

Field ID  \
Primary category                        Field title                                                    
Diet (100052)                           Never eat eggs, dairy, wheat, sugar                     6144   
                                        Never eat eggs, dairy, wheat, sugar (pilot)            10855   
Education (100063)                      Qualifications                                          6138   
                                        Qualifications (pilot)                                 10722   
Employment (100064)                     Current employment status                               6142   
                                        Transport type for commuting to job workplace           6143   
Eyesight (100041)                       Reason for glasses/contact lenses                       6147   
                                        Eye problems/disorders                                  6148   
Family history (100034)                 Illnesses of father                                    20107   
                                        Illnesses of mother                                    20110   
                                        Illnesses of siblings                                  20111   
                                        Illnesses of adopted father                            20112   
                                        Illnesses of adopted mother                            20113   
                                        Illnesses of adopted siblings                          20114   
Household (100066)                      Gas or solid-fuel cooking/heating                       6139   
                                        Heating type(s) in home                                 6140   
                                        How are people in household related to participant      6141   
                                        Gas or solid-fuel cooking/heating (pilot)              10860   
Medical conditions (100044)             Vascular/heart problems diagnosed by doctor             6150   
                                        Fractured bone site(s)                                  6151   
                                        Blood clot, DVT, bronchitis, emphysema, asthma,...      6152   
Medication (100045)                     Medication for cholesterol, blood pressure, dia...      6153   
                                        Medication for pain relief, constipation, heart...      6154   
                                        Vitamin and mineral supplements                         6155   
                                        Medication for cholesterol, blood pressure or d...      6177   
                                        Mineral and other dietary supplements                   6179   
                                        Medication for pain relief, constipation, heart...     10004   
                                        Medication for smoking cessation, constipation,...     10005   
                                        Vitamin and mineral supplements (pilot)                10007   
                                        Vitamin supplements (pilot)                            10723   
                                        Other dietary supplements (pilot)                      10854   
Mental health (100060)                  Illness, injury, bereavement, stress in last 2 ...      6145   
                                        Manic/hyper symptoms                                    6156   
                                        Illness, injury, bereavement, stress in last 2 ...     10721   
Mouth (100046)                          Mouth/teeth dental problems                             6149   
                                        Mouth/teeth dental problems (pilot)                    10006   
Other sociodemographic factors (100067) Attendance/disability/mobility allowance                6146   
Pain (100048)                       

In [9]:
for index, row in multi_choice.iterrows():
    print(index[1])
    array_cols = [col for col in pruned_cols if int(col.split('.')[0]) == row['Field ID']]
    array_df = data[['eid'] + array_cols]
    # Create missing category and assign Na to this as well as -1 (Do not know) -3 (Prefer not to answer), -7 (None of the above)
    # Only consider first arrayed column as if missing in first will be missing for all
    if "Missing" not in array_df[array_df.columns[1]].cat.categories:
        array_df.loc[:, array_df.columns[1]] = array_df[array_df.columns[1]].cat.add_categories("Missing")
    array_df.loc[:, array_df.columns[1]] = array_df[array_df.columns[1]].fillna("Missing")
    array_df.loc[:, array_df.columns[1]] = array_df[array_df.columns[1]].replace({'-1': 'Missing', '-3': 'Missing', '-7': 'Missing'})

    # Stack all array cols
    onehot_col_name_root = f"{row['Field ID']}.0"
    stacked = array_df.melt(id_vars='eid', value_name=onehot_col_name_root).drop(columns=['variable'])
    # One hot encode stacked df. Use _ separator (rather than usual '.') between field name and choice to enable identification/filtering of these OHE'd multi-choice columns if required
    onehot = pd.get_dummies(stacked, prefix_sep='_')
    # Aggregate back to eid level
    onehot = onehot.groupby('eid').sum().reset_index()
    # Join onto main dataframe and remove old array cols
    data = data.drop(columns=array_cols)
    data = pd.merge(data, onehot, on='eid', how='left')

Never eat eggs, dairy, wheat, sugar


/tmp/slurm-33721674/ipykernel_1230926/1499992851.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['5', '5', '5', '5', '5', ..., '5', '5', '5', '5', '4']
Length: 502180
Categories (7, object): ['2', '4', '5', '1', '3', '-3', 'Missing']' has dtype incompatible with category, please explicitly cast to a compatible dtype first.
  array_df.loc[:, array_df.columns[1]] = array_df[array_df.columns[1]].cat.add_categories("Missing")
/tmp/slurm-33721674/ipykernel_1230926/1499992851.py:10: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  array_df.loc[:, array_df.columns[1]] = array_df[array_df.columns[1]].replace({'-1': 'Missing', '-3': 'Missing', '-7': 'Missing'})
/tmp/slurm-33721674/ipykernel_1230926/1499992851.p

Never eat eggs, dairy, wheat, sugar (pilot)


/tmp/slurm-33721674/ipykernel_1230926/1499992851.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[NaN, NaN, NaN, NaN, NaN, ..., NaN, NaN, NaN, NaN, NaN]
Length: 502180
Categories (7, object): ['5', '4', '2', '3', '1', '-3', 'Missing']' has dtype incompatible with category, please explicitly cast to a compatible dtype first.
  array_df.loc[:, array_df.columns[1]] = array_df[array_df.columns[1]].cat.add_categories("Missing")
/tmp/slurm-33721674/ipykernel_1230926/1499992851.py:10: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  array_df.loc[:, array_df.columns[1]] = array_df[array_df.columns[1]].replace({'-1': 'Missing', '-3': 'Missing', '-7': 'Missing'})
/tmp/slurm-33721674/ipykernel_1230926/1499992851.p

Qualifications


...

/tmp/slurm-33721674/ipykernel_1230926/1499992851.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[NaN, NaN, NaN, NaN, '3', ..., NaN, '3', NaN, NaN, NaN]
Length: 502180
Categories (8, object): ['3', '4', '-7', '1', '2', '-1', '-3', 'Missing']' has dtype incompatible with category, please explicitly cast to a compatible dtype first.
  array_df.loc[:, array_df.columns[1]] = array_df[array_df.columns[1]].cat.add_categories("Missing")
/tmp/slurm-33721674/ipykernel_1230926/1499992851.py:10: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  array_df.loc[:, array_df.columns[1]] = array_df[array_df.columns[1]].replace({'-1': 'Missing', '-3': 'Missing', '-7': 'Missing'})
/tmp/slurm-33721674/ipykernel_1230926/14999

Why reduced smoking


/tmp/slurm-33721674/ipykernel_1230926/1499992851.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['4', NaN, NaN, NaN, NaN, ..., NaN, NaN, '-7', NaN, NaN]
Length: 502180
Categories (8, object): ['4', '3', '1', '2', '-7', '-3', '-1', 'Missing']' has dtype incompatible with category, please explicitly cast to a compatible dtype first.
  array_df.loc[:, array_df.columns[1]] = array_df[array_df.columns[1]].cat.add_categories("Missing")
/tmp/slurm-33721674/ipykernel_1230926/1499992851.py:10: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  array_df.loc[:, array_df.columns[1]] = array_df[array_df.columns[1]].replace({'-1': 'Missing', '-3': 'Missing', '-7': 'Missing'})
/tmp/slurm-33721674/ipykernel_1230926/1499

Leisure/social activities


/tmp/slurm-33721674/ipykernel_1230926/1499992851.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['-7', '3', '-7', '4', '-7', ..., '2', '-7', '-7', '3', '-7']
Length: 502180
Categories (8, object): ['-7', '1', '2', '3', '4', '5', '-3', 'Missing']' has dtype incompatible with category, please explicitly cast to a compatible dtype first.
  array_df.loc[:, array_df.columns[1]] = array_df[array_df.columns[1]].cat.add_categories("Missing")
/tmp/slurm-33721674/ipykernel_1230926/1499992851.py:10: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  array_df.loc[:, array_df.columns[1]] = array_df[array_df.columns[1]].replace({'-1': 'Missing', '-3': 'Missing', '-7': 'Missing'})
/tmp/slurm-33721674/ipykernel_1230926/

## Pick first array for all other field types
Not a perfect option, as some fields have measurements that should probably be averaged - but will do for now!

In [10]:
arrayed_fields = display_arrayed_fields(pruned_cols)
other_dtypes = arrayed_fields[arrayed_fields['Field type'] != 'Multi choice']
other_dtypes

Field ID  \
Primary category                 Field title                                                    
Blood pressure (100011)          Systolic blood pressure, manual reading                   93   
                                 Diastolic blood pressure, manual reading                  94   
                                 Pulse rate (during blood-pressure measurement)            95   
                                 Time since interview start at which blood press...        96   
                                 Pulse rate, automated reading                            102   
                                 Diastolic blood pressure, automated reading             4079   
                                 Systolic blood pressure, automated reading              4080   
                                 Method of measuring blood pressure                      4081   
Blood sample collection (100002) Time blood sample collected                             3166   
                                 Blood sample #, note contents                          20049   
Home locations (150)             Date first recorded at location [obsolete]             22700   
Lights pattern memory (100028)   Number of columns displayed (pilot)                    10142   
                                 Number of rows displayed (pilot)                       10143   
                                 Time taken to complete lights test (pilot)             10144   
                                 Pattern of lights displayed (pilot)                    10145   
                                 Pattern of lights as remembered (pilot)                10146   
Spirometry (100020)              Result ranking                                          3059   
                                 Time of blow measurement                                3060   
                                 Acceptability of each blow result                       3061   
                                 Forced vital capacity (FVC)                             3062   
                                 Forced expiratory volume in 1-second (FEV1)             3063   
                                 Peak expiratory flow (PEF)                              3064   
                                 Ordering of blows                                       3065   
                                 Result ranking (pilot)                                 10691   
                                 Acceptability of each blow result (pilot)              10693   
                                 Forced vital capacity (FVC) (pilot)                    10694   
                                 Forced expiratory volume in 1-second (FEV1) (pi...     10695   
                                 Peak expiratory flow (PEF) (pilot)                     10696   
                                 Acceptability of each blow result (text)               20031   
                                 Acceptability of each blow result (text) (pilot)       20032   

                                                                                        Field type  \
Primary category                 Field title                                                         
Blood pressure (100011)          Systolic blood pressure, manual reading                   Integer   
                                 Diastolic blood pressure, manual reading                  Integer   
                                 Pulse rate (during blood-pressure measurement)            Integer   
                                 Time since interview start at which blood press...        Integer   
                                 Pulse rate, automated reading                             Integer   
                                 Diastolic blood pressure, automated reading               Integer   
                                 Systolic blood pressure, automated reading                Integer   
                                 Method of measuring blood pre

In [11]:
# Get all columns for other data types
other_dtypes_cols = [col for col in pruned_cols if int(col.split('.')[0]) in other_dtypes['Field ID'].tolist()]
# Get all non-first arrays and remove from main dataframe 
not_first_array = [col for col in other_dtypes_cols if int(col.split('.')[2]) > 0]
data = data.drop(columns=not_first_array)

In [12]:
# Check that no arrayed columns remain (excluding one-hot-encoded fields that contain '_')
display_arrayed_fields([col for col in data.columns[1:] if '_' not in col])

,,Field ID,Field type,Number of arrays
Primary category,Field title,,,


# Final processing

## Remove inappropriate data types
String and datetime

In [13]:
fields_metadata = pd.read_csv("../data/raw/ukb_metadata/field.txt", sep="\t") # Schema 1

fields_to_remove = []

# Strings
fields_to_remove.extend(fields_metadata.loc[fields_metadata['value_type'] == 41, 'field_id'].tolist())

# Date
fields_to_remove.extend(fields_metadata.loc[fields_metadata['value_type'] == 51, 'field_id'].tolist())

# Time
fields_to_remove.extend(fields_metadata.loc[fields_metadata['value_type'] == 61, 'field_id'].tolist())

data = data[['eid'] + [col for col in data.columns[1:] if int(col.split('.')[0]) not in fields_to_remove]]

## Handle missing values
-1: Do not know
-3: Prefer not to answer
-7: None of the above

Make these all NaN

In [14]:
missing_values = ['-1', '-3', '-7']

data[data.select_dtypes(['category']).columns] = data.select_dtypes(['category']).apply(
    lambda col: col.map(lambda x: np.nan if x in missing_values else x).astype('category')
)

# View final included categories

In [15]:
pd.set_option('display.max_rows', None)
cat_tree, pretty_df = display_included_cats(data.columns[1:])
pretty_df.to_excel('../output/Final included categories.xlsx')
pretty_df

Empty DataFrame
Columns: []
Index: [(Assessment centre (100000), Recruitment (100021), Reception (100024) ✅, , , ), (Assessment centre (100000), Recruitment (100021), Consent (100023) ✅, , , ), (Assessment centre (100000), Recruitment (100021), Conclusion (100022), , , ), (Assessment centre (100000), Recruitment (100021), Consent for imaging (100119), , , ), (Assessment centre (100000), Touchscreen (100025), Sociodemographics (100062), Household (100066) ✅, , ), (Assessment centre (100000), Touchscreen (100025), Sociodemographics (100062), Employment (100064) ✅, , ), (Assessment centre (100000), Touchscreen (100025), Sociodemographics (100062), Education (100063) ✅, , ), (Assessment centre (100000), Touchscreen (100025), Sociodemographics (100062), Ethnicity (100065) ✅, , ), (Assessment centre (100000), Touchscreen (100025), Sociodemographics (100062), Other sociodemographic factors (100067) ✅, , ), (Assessment centre (100000), Touchscreen (100025), Lifestyle and environment (100050), Physical activity (100054) ✅, MET Scores (54) ✅, ), (Assessment centre (100000), Touchscreen (100025), Lifestyle and environment (100050), Electronic device use (100053) ✅, , ), (Assessment centre (100000), Touchscreen (100025), Lifestyle and environment (100050), Sleep (100057) ✅, , ), (Assessment centre (100000), Touchscreen (100025), Lifestyle and environment (100050), Smoking (100058) ✅, , ), (Assessment centre (100000), Touchscreen (100025), Lifestyle and environment (100050), Diet (100052) ✅, , ), (Assessment centre (100000), Touchscreen (100025), Lifestyle and environment (100050), Alcohol (100051) ✅, , ), (Assessment centre (100000), Touchscreen (100025), Lifestyle and environment (100050), Sun exposure (100055) ✅, , ), (Assessment centre (100000), Touchscreen (100025), Lifestyle and environment (100050), Sexual factors (100056) ✅, , ), (Assessment centre (100000), Touchscreen (100025), Early life factors (100033) ✅, , , ), (Assessment centre (100000), Touchscreen (100025), Family history (100034) ✅, , , ), (Assessment centre (100000), Touchscreen (100025), Psychosocial factors (100059), Social support (100061) ✅, , ), (Assessment centre (100000), Touchscreen (100025), Psychosocial factors (100059), Mental health (100060) ✅, , ), (Assessment centre (100000), Touchscreen (100025), Health and medical history (100036), Eyesight (100041) ✅, , ), (Assessment centre (100000), Touchscreen (100025), Health and medical history (100036), Mouth (100046) ✅, , ), (Assessment centre (100000), Touchscreen (100025), Health and medical history (100036), General health (100042) ✅, , ), (Assessment centre (100000), Touchscreen (100025), Health and medical history (100036), Breathing (100037) ✅, , ), (Assessment centre (100000), Touchscreen (100025), Health and medical history (100036), Claudication and peripheral artery disease (100038) ✅, , ), (Assessment centre (100000), Touchscreen (100025), Health and medical history (100036), Pain (100048) ✅, , ), (Assessment centre (100000), Touchscreen (100025), Health and medical history (100036), Chest pain (100039) ✅, , ), (Assessment centre (100000), Touchscreen (100025), Health and medical history (100036), Cancer screening (100040) ✅, , ), (Assessment centre (100000), Touchscreen (100025), Health and medical history (100036), Operations (100047) ✅, , ), (Assessment centre (100000), Touchscreen (100025), Health and medical history (100036), Medical conditions (100044) ✅, , ), (Assessment centre (100000), Touchscreen (100025), Health and medical history (100036), Medication (100045) ✅, , ), (Assessment centre (100000), Touchscreen (100025), Health and medical history (100036), Hearing (100043) ✅, , ), (Assessment centre (100000), Touchscreen (100025), Sex-specific factors (100068), Male-specific factors (100070) ✅, , ), (Assessment centre (100000), Touchscreen (100025), Sex-specific factors (100068), Female-specific factors (100069) ✅, , ), (Assessment centre (100000), Cognitive function (100026), Reaction time (

# Save

In [16]:
data.to_feather("../data/processed/features_processed.feather")